In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [24]:
df=pd.read_excel('/content/dataset.xlsx')


In [25]:
print(df.head())

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  
3                     0.167   21        0  
4                     2.288   33        1  


In [26]:
cols_with_zeros = ["Glucose", "BloodPressure", "SkinThickness", "Insulin", "BMI"]
df[cols_with_zeros] = df[cols_with_zeros].replace(0, np.nan)
#NaN clearly tells Python/ML libraries “this is missing”.
#Libraries like scikit-learn can handle NaN using imputers (SimpleImputer, KNNImputer etc.).
#If you keep 0 as is, your model will think patients can actually have Glucose=0, which adds noise and hurts accuracy.

Common Imputation Strategies

mean→ fill missing values with column mean

median → fill missing values with column median (robust against outliers 👍)

most_frequent → fill with mode (common for categorical data)

constant → replace with a fixed value you set

The mean is sensitive → one extreme value can distort it a lot.

The median is robust → it only cares about the order of values, not how far they are.


In [27]:
#Median is robust because it ignores extremes.
#Use mean only if the distribution is roughly normal.
#Use mode for categorical features.

# Step 1: Create the imputer (median strategy)
imputer = SimpleImputer(strategy="median")

# Step 2: Fit on the dataset (calculate medians for each column)
imputer.fit(df[cols_with_zeros])

# You can check what medians were learned
print("Calculated medians:", imputer.statistics_)

# Step 3: Transform (replace NaN with those medians)
df[cols_with_zeros] = imputer.transform(df[cols_with_zeros])

Calculated medians: [117.   72.   29.  125.   32.3]


In [28]:
import pickle

# Save the imputer
with open("imputer.pkl", "wb") as f:
    pickle.dump(imputer, f)


In [29]:
df.to_csv('preprocessed_dataset_without_scaling.csv', index=False)

In [30]:
df_capped=df.copy()

for col in df_capped.columns[:-1]: #exclude target column
  col_mean=df_capped[col].mean()
  col_std=df_capped[col].std()
  upper_limit=col_mean+(3*col_std)
  lower_limit=col_mean-(3*col_std)
  df_capped[col]=np.where(df_capped[col]>upper_limit,upper_limit,df_capped[col])
  df_capped[col]=np.where(df_capped[col]<lower_limit,lower_limit,df_capped[col])
  #np.where(condition, value_if_true, value_if_false)
  #for i in range(len(df_capped)):
    # if df_capped.loc[i,col]>upper_limit:
    # df_capped.loc[i,col]=upper_limit
    # elf df_capped.loc[i,col]<lower_limit:
    # df_capped.loc[i,col]=lower_limit

In [31]:
df_capped.to_csv('outlier_refined.csv',index=False)

In [32]:
X=df_capped.drop("Outcome", axis=1)
#input features
y=df_capped["Outcome"]
#output features

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [34]:
#(stratify=y): Ensures that the proportion of each class in train and test is the same as in the original dataset.
#This is not eliminating randomness, it’s still random within each class. So your model still sees varied samples, but the classes are fairly represented.
#Scaling means changing the range of your features so that they are on a similar scale.
# This is important because many machine learning algorithms (like logistic regression, SVM, KNN, neural networks, etc.) are sensitive to the magnitude of input features.
#z=(x-mean)/standard_deviation
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [35]:
print("X_train_scaled shape:", X_train_scaled.shape)
print("X_test_scaled shape:", X_test_scaled.shape)


X_train_scaled shape: (614, 8)
X_test_scaled shape: (154, 8)


In [36]:
print(X.columns)

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age'],
      dtype='object')


In [37]:
X_train_scaled_df=pd.DataFrame(X_train_scaled,columns=X.columns)
X_test_scaled_df=pd.DataFrame(X_test_scaled,columns=X.columns)


In [38]:
# Save training features
X_train_scaled_df.to_csv("X_train_scaled.csv", index=False)

In [39]:
# Save testing features
X_test_scaled_df.to_csv("X_test_scaled.csv", index=False)

In [40]:
# Save labels (targets)
y_train.to_csv("y_train.csv", index=False)
y_test.to_csv("y_test.csv", index=False)

In [41]:
import pickle

In [42]:
#By dumping (saving) it into a file called scaler.pkl
#you can later load the exact same scaler when making predictions.
with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

In [43]:
#This saves the list of column names used during training
with open("feature_columns.pkl", "wb") as f:
    pickle.dump(X.columns.tolist(), f)